# Training & Validation notebook

This notebook walks through our training and validation scripts.

### Setting up the environment
Create a new virtual environment using `venv` or if already created, move on to the next part (skip this if running on Google colab).

In [ ]:
# !python3 -m venv venv
# !source venv/bin/activate

### Cloning the repo
Clone the repo and install the required packages defined in `requirements.txt`.

In [1]:
%%capture
!git clone https://github.com/arnavrneo/torchFlow.git
%cd torchFlow
!pip install -r requirements.txt

Run the following shell script to (skip this if the data is already arranged):
- download the `yolo` and `onnx` model checkpoints to the `models` directory;
- download the tiled datasets in their respective directories.

In [4]:
%%capture
!./get-data.sh

In [5]:
!python config/config.py # for setting up the dataset directorys

## Training

For training, we have trained the model by tiling the dataset into the following sizes:
- 256 x 256
- 512 x 512
- 1280 x 1280

and recursively training the model on next tiled dataset and then finally training at 3200 x 2600 size on the original dataset.

The base model used: `yolov8l.pt`.

For replicating training process, keep changing:
- the model sizes acc. to the tile size (keep the params as it is) in the `train-config.yaml` file.
![epochs-sizes.png](assets/epochs-sizes.png)


- and the dataset directory path in the `dataset.yaml`, i.e.

![dataset-yaml.png](assets/dataset-yaml.png)

- and the model checkpoints.

The arguments:
- `-m`: model path

In [ ]:
!python train.py -m yolov8l.pt

## Validation

Validation will be done automatically during the training step, but for manual validation, we provide the `val.py` script.
- We run the validation at `3232` size. The values can be changed in the `val-config.yaml`.

The arguments:
- `-m`: model path

In [ ]:
!python val.py -m /kaggle/working/torchFlow/models/torchFlow-ckpt.pt